# Reading In and Cleaning California Alcohol Outlet Data

## Introduction
The California Alcohol Beverage Control maintains a [dataset](http://www.abc.ca.gov/datport/DataExport.html) of the current list of alcohol licenses in the state, which is exported weekly. This includes a wide range of license types for various on-premise or off-premise alcohol, each of which carries different privileges (beer only, winegrower, restaurant, etc). A single location can hold multiple license types (a wine grower can also sell wine, a restaurant can also have a bar, etc). The research organization I work for uses this type of data to study spatial relationships between sources of alcohol and alcohol related problems. 

Because of the way that this data is formatted, there are a number of steps that need to be taken along the way before it finds itself in a usable form. ```Pandas```, ```Geopandas```, and ```Geopy``` make this process relatively easy, as I demonstrate below. In the following notebook, I will:

1. Download the most recent export of California alcohol license data
2. Read in the fixed-with text file into Pandas
3. Aggregate license-level data to location-level data, identifying locations of three types of active alcohol outlets: off-premise outlets (liquor stores), bars, and restaurants. 
4. Geocode and get the lat/long coordinates of a subset of the data - all bars in San Francisco
5. Convert lat / long coordinates to projected GeoSeries as part of GeoDataFrame
6. Overlay point locations with a shapefile of SF neighborhoods to look at bar densities
7. Make a simple map using the built-in plotting feature in GeoPandas
8. Export the data as a shapefile for future use


In [1]:
#import necessary modules
%matplotlib inline
import zipfile, os, urllib
import pandas as pd
import numpy as np
import geopandas as gpd
import geopy
import os
import shapely
from shapely.geometry import Point
import geopandas.tools
import matplotlib.pyplot as plt



Bad key "text.kerning_factor" on line 4 in
/Users/sankeerth/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


ModuleNotFoundError: No module named 'geopandas'

## Download Raw Alcohol License Data
Below I create a data directory and use ```urllib``` to dowload the data export and then ```zipfile``` to unzip it (if this hasn't been done already). I extract and print the name of the raw file which is an LST file. An LST file is just a plain text file that gets exported from some database programs.

In [ ]:
if not os.path.exists('Data'):
    os.mkdir('Data')

URL='https://www.abc.ca.gov/datport/ABC_Data_Export.zip'
zip_file='Data/'+os.path.basename(URL)
if not os.path.exists(zip_file):
    urllib.urlretrieve(URL,zip_file)

z=zipfile.ZipFile(zip_file)

rawfile=[Item for Item in z.namelist()][0]
if not os.path.exists('Data/'+rawfile):
    z.extract(rawfile, 'Data')
rawfile='Data/'+rawfile
print rawfile

## Read in Data to Pandas
The ABC website contains (link above) contains a document that length of each field in the text file. The text file is in fixed-with format, which means that field lengths need to be specified in order for the data to be parsed correctly. Below I create lists that contain the length of each field as well as the name of each field.

In [ ]:
col_lengths=[2,8,3,8,11,11,8,3,1,2,4,2,50,50,50,25,2,10,50,50,50,25,2,10,16,7]
col_names=['LicenseType','FileNum','LicenseOrApp','Status','OrigIssueDate','ExpirDate',\
           'FeeCodes','DupCounts','MasterIndicator','TermMonths','GeoCode','DistrictCode',\
           'pName','pAddress1','pAddress2','pCity','pState','pZip','DBAName','mAddress1',\
           'mAddress2','mCity','mState','mZip','pCounty','CensusTract']



Pandas can read in fixed width format files, but prefers to have the columns specified as a range of character positions, rather than a width. This is more flexible because it allows for gaps in the file that don't represent any data. Because there are no such gaps in the ABC data, the columns all butt up against each other and I can write a simple function that converts a list of widths to a list of ranges.

In [ ]:
def length_to_range(length_list):
    col_ranges=[]
    i=0
    for length in length_list:
       col_range=(i,length+i)
       col_ranges.append(col_range)
       i=i+length
    return col_ranges

col_ranges=length_to_range(col_lengths)
print col_ranges

I am then ready to read in the text file to ```Pandas``` using the 'colspecs' argument to specify these newly calculated column ranges, and 'names' to specify column names. I subset the data to include relevant columns and set the index as the identifier 'FileNum', which is an identifier for each unique license holder (not license).

In [ ]:
df=pd.read_fwf(rawfile, colspecs=col_ranges, names=col_names)
df=df[['LicenseType','FileNum','Status','DBAName','pName','pAddress1','pCity','pState','pZip','pCounty']].set_index('FileNum')
df.head()

## Aggregating License Data to Outlet Data
The index "FileNum" is not unique as it represents license holders, which as mentioned can hold multiple licenses. As we see below, there are almost 20,000 more licenses than license holders.

In [ ]:
print len(df), len(df.index.unique())

Therefore, within each "FileNum" group, license type and license status will be unique, but the other fields (Name, Address, etc.) should all be the same. I will first group by file number to get the unique locations, and will then join identifiers for bar, restaurant, and off-premise to this dataset. The dataframe below represents the unique set of license locations.

In [ ]:
outlet_data=df[[x for x in df.columns if x not in ['LicenseType','Status']]].groupby(level='FileNum').first()
outlet_data.head()

The next step is then to determine which of these unique locations fall into one of the 3 outlet categories. We also need to be aware of the fact that not all licenses in the dataset are active licenses - a single location can hold an active off-sale license and an expired bar license, for example. The list of license statuses are shown below. I choose to consider any license listed as "ACTIVE" as being an active license.

In [ ]:
df.Status.value_counts()

The ABC website also contains descriptions of license type numbers found [here](http://www.abc.ca.gov/permits/licensetypes.html). It is fairly straightforward to decide which license numbers fall into each of the 3 categories - off-premise, restaurant, bar. I create a dictionary below that contains the lists of license types in each categories.

In [ ]:
outlet_types={'OffPremise':[20,21], 'Rest':[41,47],'Bar':[40,42,48,61,75]}

Now I will loop through each of the 3 outlet categories to check for active codes in each category's list of types. I query the license data to identify those that have a license type in each set of codes, and a status that is active. I then group by the file number specifiying that ```any``` of the licenses within a location can satisfy this query. This results in a boolean indexed by ```FileNum``` which is then just converted to an integer and added to the location level dataframe. As you can see below, our dataframe now has fields that indicate whether or not a location holds a license in one of those 3 categories we defined.

In [ ]:
for outlet_type in outlet_types:
    codes=outlet_types[outlet_type]
    active_lictype=((df['LicenseType'].isin(codes))&(df['Status']=='ACTIVE')).groupby(level='FileNum').any()
    outlet_data[outlet_type]=active_lictype.astype(int)
outlet_data.head()

Total counts of locations in each of those 3 categories can be seen below:

In [ ]:
outlet_data[outlet_types.keys()].sum()

## Geocoding Bars in San Francisco
Now we have a dataframe that contains for each unique location in California that holds a liquor license, the outlet name, address, and indicators for whether that location is an active bar, restaurant, or off-premise outlet (as defined by my previous license type categorizations). The piece of useful information would be to have the actual point locations of these locations. For this example, I focus only on bars within San Francisco. I write a function that wraps ```geopy``` and geocodes a Pandas series of addresses.

First I subset my dad to get only the bars within San Francisco, of which there are 433 according to this data.

In [ ]:
sf_bars=outlet_data[(outlet_data.Bar==1)&(outlet_data.pCity=='SAN FRANCISCO')].drop(['Bar','Rest','OffPremise'],1)
print len(sf_bars)
sf_bars.head()


Now I write a function that wraps ```geopy```, which itself wraps the geocoding API's of several other sources. I choose to use a locator that uses the Google API. The output of the function is a Pandas dataframe that contains the cleaned match address, x and y coordinates, and a field that identifies the type of match that Google made (rooftop, approximate, range-interpolated, etc). 

In [ ]:
def geocode_google_add(addresses, key=None):
    google_locator=geopy.geocoders.GoogleV3(api_key=key) #define locator using google api
    outputcols=['match_addr', 'x','y','loc_type'] #list columns for output dataframe
    match_result=pd.DataFrame(columns=outputcols) #define output dataframe
    for i,row in addresses.iteritems(): # loop through addresses in series
        try: 
            location = google_locator.geocode(row) #try geocoding the address
            if location: #if it geocodes, add it's location information to the dataframe
                outputvals=[location.address,location.longitude, location.latitude, location.raw['geometry']['location_type']]
                match_result.loc[i]=outputvals
            else: #if it does not find a location, leave the location fields blank
                match_result.loc[i]=[None]*len(outputcols)
        except Exception, e: # catch geocoding errors and print an error message
            match_result.loc[i]=[None]*len(outputcols)
            print 'Failed at ID '+str(i)
            print e
            break
    return match_result

The current dataset has separate fields for address components. I concatenate these into one address field that will be used as the input to the geocoding function.

In [ ]:
bar_addresses=sf_bars['pAddress1']+', '+sf_bars['pCity']+', '+sf_bars['pState']+' '+sf_bars['pZip']
bar_addresses.head()

If the data has not been geocoded, I geocode the data using the function I defined and export it. Calculating frequencies on the location match type, we can see Google was able to geocode 93% of the input addresses to a rooftop location, 6% were range interpolated within a street block, and about 1% were somewhat more approximate locations.

In [ ]:
geocode_output="Data/sf_bar_geocoded.p"
if not os.path.exists(geocode_output):
    sf_bars_geocoded=geocode_google_add(bar_addresses)
    sf_bars_geocoded.to_pickle(geocode_output)
else:
    sf_bars_geocoded=pd.read_pickle(geocode_output)
print sf_bars_geocoded.loc_type.value_counts()/float(len(sf_bars_geocoded))

When using the Google Geocoding API, I generally consider those that match as "ROOFTOP" or "RANGE_INTERPOLATED" to be valid matches, but do not consider "APPROXIMATE" or "GEOMETRIC_CENTER". The addresses geocoded quite well (99%), so I'm not going to worry about those that did not match.

In [ ]:
sf_bars_matched=sf_bars_geocoded[sf_bars_geocoded.loc_type.isin(['ROOFTOP','RANGE_INTERPOLATED'])]
sf_bars_matched.head()

## Converting Geocoded Lat / Long Coordinates to GeoDataFrame
The next step is to convert the lat/long coordinates to ```shapely``` geometry objects, which will then be built into a Geopandas GeoDataFrame. I've already imported ```Point``` from the ```shapely``` geometry set, so I can just go ahead and write a function that pulls the x and y coordinates from each geocoded address. I apply this function to the entire series and then get a series of ```shapely``` geometries as my output.

In [ ]:
geometry=sf_bars_geocoded.apply(lambda row:Point(row['x'],row['y']),1)
geometry.head()

Now I want to create a GeoDataframe that contains the attributes of the unique locations and the geometry that was just calculated from the x/y coordinates of the geocoded data. Because the geometry and the dataframe are both indexed the same (by FileNum) I can just specify the geometry as I define the GeoDataFrame. Also, since ```shapely``` geometries do not hold any information on the coordinate system they represent, I need to define the coordinate system as I create the GeoDataframe. I do this using EPSG codes. EPSG 4326 is code for WGS 84, which is the standard lat / long unprojected coordinate system. Also, within the same line, after defining the coordinate system of the ```shapely``` geometries, I convert to the projected coordinate system NAD 83 California Teale Albers (EPSG 3310). Documentation on spatial reference systems can be found [here](http://spatialreference.org/).

Now I have a GeoDataFrame of all of the licensed bars in San Francisco projected in California Teale Albers.

In [ ]:
sf_bars_loc=gpd.GeoDataFrame(data=sf_bars, geometry=geometry, crs={'init': 'epsg:4326'}).to_crs(epsg=3310)
sf_bars_loc.head()

## Overlay Bar Locations with SF Neighborhoods
Next, I read in a shapefile of San Francisco neighborhood boundaries that I downloaded from the city's open data portal. I read the shapefile in as a GeoDataFrame, convert to the geometry to the same coordinate system that the bar data is in (EPSG 3310), and index it by the neigborhood name

In [ ]:
neighs=gpd.read_file('Data/planning_neighborhoods.shp').to_crs(sf_bars_loc.crs).set_index('neighborho').sort_index()
neighs.head()

Then in 1 line of code I can overlay and get the counts of bars in each of the neighborhoods. I spatially join neighborhoods to bars, aggregate by neighborhood name while calculating the count, and then reindex by the neighborhood index to capture those neighborhoods without any bars.

In [ ]:
neigh_bar_count=gpd.tools.sjoin(sf_bars_loc, neighs.reset_index()).groupby('neighborho').size().reindex(neighs.index).fillna(0)
neigh_bar_count

However, bar counts aren't particularly useful because there is such variation in neighborhood size. I instead calculate the number of bars per square mile in each neighborhood by dividing the series of neighborhood counts by a series of neighborhood area. Note that I need to convert from square meters (the units of EPSG 3310) to square miles. I also sort by bar density and the results are what we would expect to see - Downtown, Chinatown, and the Financial District are among the areas with the greatest bar density, while Sea Cliff, Golden Gate Park, and Twin Peaks are among the areas with the lowest bar density.

In [ ]:
bars_p_sqmi=(neigh_bar_count/(neighs.area/2.59e+6)).sort_values(inplace=False,ascending=False)
bars_p_sqmi.name='BarsPSqMi'
bars_p_sqmi

## Plot Bars and Neighborhoods using GeoPandas

Now I will use the built-in plotting features of GeoPandas to make a quick map overlaying bar point locations on neighborhood boundaries symbolized by bar density. You'll notice below that I actually plot a subset of the bars. This is because two of the bars are located at the airport which although managed by San Francisco is technically not in San Francisco city boundaries. To clean the map up, I get rid of these bars by selecting only those that intersect the union of neighborhoods. 

And when plotting the neighborhoods, I first need to create a GeoDataFrame composed of the data from the bar density Series and the geometry from the neighborhood GeoDataFrame. I specify that I want to symbolize by the field "BarsPSqMi" broken into quantiles, using a red colormap.

In [ ]:
sf_bars_loc[sf_bars_loc.intersects(neighs.unary_union)].plot()
gpd.GeoDataFrame(bars_p_sqmi,geometry=neighs.geometry,crs=neighs.crs).plot(column='BarsPSqMi', scheme='QUANTILES', colormap='OrRd')
fig=plt.xticks(rotation=40)

## Export Bar GeoDataframe to Shapefile

GeoPandas makes this very easy!

In [ ]:
sf_bars_loc.to_file('Data/sf_bar_locations.shp')